In [1]:
!pip install sentence_transformers
!pip install pyngrok
!pip install flask_cors
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.4/130.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 101.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from sentence_transformers import SentenceTransformer, CrossEncoder
import pandas as pd
from sentence_transformers import util
import torch
import gradio as gr
from flask import Flask, request, jsonify
from pyngrok import ngrok
from flask_cors import CORS
import requests  # Thư viện để gọi API HTTP

# Models

In [3]:
model = SentenceTransformer("/kaggle/input/m/ictunivers/halong_embedding/transformers/default/1/checkpoint-2500")

In [4]:
model_name = "BAAI/bge-reranker-v2-m3"
cross_encoder = CrossEncoder(model_name, num_labels=1, max_length=1024)

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [5]:
import logging
logging.disable(logging.WARNING)

# Corpus

In [6]:
corpus = pd.read_csv("/kaggle/input/corpus-after-preprocessed-18de/corpus_after_preprocessed_18Dec.csv")
corpus.head()

,text,cid
0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",0
1,"Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân cấ...",1
2,Tiêu chuẩn của các thành viên thuộc lực lượng ...,2
3,"Nhiệm vụ của lực lượng tuần tra, canh gác đê C...",3
4,"Phù hiệu của lực lượng tuần tra, canh gác đê P...",4


In [7]:
corpus.head()

,text,cid
0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",0
1,"Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân cấ...",1
2,Tiêu chuẩn của các thành viên thuộc lực lượng ...,2
3,"Nhiệm vụ của lực lượng tuần tra, canh gác đê C...",3
4,"Phù hiệu của lực lượng tuần tra, canh gác đê P...",4


In [8]:
# import pandas as pd
# import h5py
# from tqdm import tqdm
# import numpy as np


# # Extract texts 
# texts = corpus["context"].tolist() 
# cids = corpus['cid'].tolist()


# batch_size = 128  # Adjust this based on your system's memory
# embeddings = []
# model = model.to('cuda')
# print(model.device)
# print("Encoding text...")
# for start_idx in tqdm(range(0, len(texts), batch_size)):
#     batch_texts = texts[start_idx:start_idx + batch_size]
#     batch_embeddings = model.encode(batch_texts, convert_to_numpy=True, show_progress_bar=False)
#     embeddings.append(batch_embeddings)

# embeddings = np.vstack(embeddings)
# print("start embeddings")
# output_file = "/kaggle/working/corpus_embeddings_halong_03.h5"  
# print(f"Saving embeddings to {output_file}...")
# with h5py.File(output_file, "w") as f:
#     f.create_dataset("embeddings", data=embeddings)
#     f.create_dataset("cids", data=np.array(cids, dtype="S")) 

# print("Embeddings saved successfully.")

# Vector embeddings call 

In [9]:
import h5py

# Đường dẫn đến tệp HDF5
file_path = "/kaggle/input/corpus-embeddings-halong/corpus_embeddings_halong.h5"

# Mở tệp HDF5
with h5py.File(file_path, "r") as f:
    # Hiển thị các nhóm trong tệp
    print("Các nhóm trong tệp HDF5:", list(f.keys()))

    # Lấy nhóm cids và embeddings
    cids = f['cids'][:]  # Lấy toàn bộ dữ liệu trong nhóm cids
    embeddings = f['embeddings'][:]  # Lấy toàn bộ dữ liệu trong nhóm embeddings

    # Lấy một mẫu từ nhóm cids và nhóm embeddings
    cid_sample = cids[0]  # Ví dụ: lấy mẫu đầu tiên từ cids
    embedding_sample = embeddings[0]  # Lấy mẫu đầu tiên từ embeddings

    # Hiển thị kết quả
    print("Sample từ nhóm cids:", cid_sample)  # Giải mã chuỗi byte
    print("Sample từ nhóm embeddings:", embedding_sample)

Các nhóm trong tệp HDF5: ['cids', 'embeddings']
Sample từ nhóm cids: b'0'
Sample từ nhóm embeddings: [-4.00290713e-02  7.38377869e-03 -2.42152214e-02  3.50992978e-02
  3.98945957e-02 -3.95338088e-02  4.71187532e-02  4.36890684e-03
  9.79055539e-02  6.11099601e-02 -6.70872480e-02  2.13058572e-02
 -4.70744669e-02  3.55103090e-02 -8.16077292e-02 -4.13716072e-03
 -5.20001771e-03  4.97970218e-03 -2.35504080e-02 -1.10263318e-01
 -5.17912321e-02  6.32728338e-02  1.60667915e-02  8.69306829e-03
  1.90416514e-03 -1.91577859e-02 -1.29759852e-02 -4.15546522e-02
 -4.02250662e-02  4.69485344e-03 -2.36048596e-03 -1.68318264e-02
 -1.29732070e-02  5.53600341e-02 -1.74243636e-02 -1.96220912e-02
 -2.02063341e-02  1.17552876e-02  5.68123683e-02 -4.30936851e-02
 -1.86395422e-02  2.71649901e-02  7.24907173e-03 -3.32809985e-02
 -9.83090978e-03  2.08635014e-02  2.78888810e-02  1.33905234e-03
  6.35454804e-02  4.16411981e-02 -1.32311992e-02 -6.83095232e-02
 -6.26494959e-02 -3.15341428e-02 -4.10749763e-02 -1.58

In [10]:
import re

def preprocess_text(text):
    # Replace newline characters with spaces
    text = ' '.join(text.splitlines())
    
    # Replace escaped newline characters with periods
    text = text.replace('\\n', '.')
    
    # Remove specific unwanted characters and patterns
    text = text.replace('/', '')
    text = text.replace("''", '')
    text = text.strip("[]")
    text = text.strip("'")
    text = text.strip('"')
    
    # Remove numbered list indicators and lettered list indicators
    text = re.sub(r'\b\d\.\s*', '', text)  # Removes patterns like '1. ', '2. ', etc.
    text = re.sub(r'\b[a-e]\)\s*', '', text)  # Removes patterns like 'a) ', 'b) ', etc.
    
    # Replace certain punctuation with periods
    text = text.replace(';', '.')
    text = text.replace(':', '.')
    
    # Remove hyphens
    text = text.replace('-', '')
    
    # Strip leading and trailing whitespace
    text = text.strip()

    text = re.sub(r'\s{2,}', ' ', text)
    # text = rdrsegmenter.word_segment(text)
    # text = ' '.join(text)

    
    return text



In [11]:
import gradio as gr
from sentence_transformers import util

# Define your question-answering function
def qa_system(question, history):
    # Replace this with your actual logic
    NGROK_SERVER_URL = "https://932a-113-23-36-6.ngrok-free.app" 
    
    query_embedding = model.encode(question)
    hits = util.semantic_search(query_embedding, embeddings, top_k=50)[0]
    cross_inp = [[preprocess_text(question), preprocess_text(corpus.iloc[hit['corpus_id']]['text'])] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)
    for idx in range(len(hits)):
        hits[idx]['cross-score'] = cross_scores[idx]
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    best_hit = hits[0]
    top_k_cross = corpus.iloc[best_hit['corpus_id']]['text']
    print('cross-encoder')
    
    # Cross-encoder refinement
    print(top_k_cross)
    
    try:
        response = requests.post(
            f"{NGROK_SERVER_URL}/ask",  # Đường dẫn tới endpoint xử lý trên server ngrok
            json={
                "question": question,
                "context": top_k_cross  # Gửi thông tin cần thiết
            }
        )
        response_data = response.json()
        
        answer = response_data["answer"]
        print(answer)
    except Exception as e:
        print(f"Error calling external server: {str(e)}")

    # Update the chat history
    history.append(("User", question))
    history.append(("System", answer))
    return history, history

# Create the Gradio interface
interface = gr.Interface(
    fn=qa_system, 
    inputs=[
        gr.Textbox(placeholder="Enter your question here...", label="Your Question"),  # User input
        gr.State([])  # To store conversation history
    ],
    outputs=[
        gr.Chatbot(label="Chat History"),  # Display conversation history
        gr.State([])  # Return updated conversation history
    ],
    title="Question Answering System",
    description="Ask your question and view the conversation history."
)

# Launch the Gradio app with a shareable link
interface.launch(share=True)


/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://3c52f6c2b849c0f997.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [43]:
print("hello")

hello


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

cross-encoder
Xử phạt người đua xe trái phép, cổ vũ đua xe trái phép Phạt tiền từ 000.000 đồng đến 000.000 đồng đối với một trong các hành vi vi phạm sau đây. Tụ tập để cổ vũ, kích động hành vi điều khiển xe chạy quá tốc độ quy định, lạng lách, đánh võng, đuổi nhau trên đường hoặc đua xe trái phép. Đua xe đạp, đua xe đạp máy, đua xe xích lô, đua xe súc vật kéo, cưỡi súc vật chạy đua trái phép trên đường giao thông. Phạt tiền từ 10.000.000 đồng đến 1000.000 đồng đối với người đua xe mô tô, xe gắn máy, xe máy điện trái phép. Phạt tiền từ 20.000.000 đồng đến 2000.000 đồng đối với người đua xe ô tô trái phép. Ngoài việc bị phạt tiền, người điều khiển phương tiện thực hiện hành vi vi phạm còn bị áp dụng các hình thức xử phạt bổ sung sau đây. Thực hiện hành vi quy định tại điểm b khoản 1 Điều này bị tịch thu phương tiện (trừ súc vật kéo, cưỡi). Thực hiện hành vi quy định tại khoản 2, khoản 3 Điều này bị tước quyền sử dụng Giấy phép lái xe từ 03 tháng đến 05 tháng và tịch thu phương tiện.
  T

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

cross-encoder
Điều 8. Điều kiện kết hôn Nam, nữ kết hôn với nhau phải tuân theo các điều kiện sau đây. Nam từ đủ 20 tuổi trở lên, nữ từ đủ 18 tuổi trở lên. Việc kết hôn do nam và nữ tự nguyện quyết định. Không bị mất năng lực hành vi dân sự. Việc kết hôn không thuộc một trong các trường hợp cấm kết hôn theo quy định tại các điểm a, b, c và d khoản 2 Điều 5 của Luật này. Nhà nước không thừa nhận hôn nhân giữa những người cùng giới tính.
  Độ tuổi kết hôn cho cả nam và nữ đều là từ đủ 20 trở lên đối với nam và từ đủ 18 tuổi trở lên, theo quy định trong Điều 8 của văn bản pháp luật đã cung cấp. 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

cross-encoder
Người chủ sở hữu, quản lý phương tiện tham gia giao thông đường bộ mà giao cho người không có giấy phép lái xe hoặc người đang trong tình trạng có sử dụng rượu, bia mà trong máu hoặc hơi thở có nồng độ cồn vượt quá mức quy định hoặc có sử dụng chất ma túy hoặc các chất kích thích mạnh khác hoặc không đủ các điều kiện khác theo quy định của pháp luật gây thương tích hoặc gây tổn hại cho sức khỏe của 01 người với tỷ lệ tổn thương cơ thể từ 31% đến 60% hoặc gây thương tích hoặc gây tổn hại cho sức khỏe của 02 người trở lên mà tổng tỷ lệ tổn thương cơ thể của những người này từ 31% đến 60%, thì bị phạt tiền từ 10.000.000 đồng đến 50.000.000 đồng hoặc phạt cải tạo không giam giữ đến 02 năm. Người phạm tội còn có thể bị phạt tiền từ 10.000.000 đồng đến 30.000.000 đồng.
  Uống rượu, bia và tham gia giao thông có thể dẫn đến việc chủ sở hữu hoặc quản lý phương tiện tham gia giao thông đường bộ bị phạt tiền từ 10.000.000 đồng đến 50.000.000 đồng hoặc cải tạo không giam giữ từ 2 nă

In [12]:
# # Lấy dữ liệu từ request.form
# # question = request.form.get('message')
# question = 'Nam bao nhiêu tuổi có thể kết hôn ? '

# NGROK_SERVER_URL = "https://7501-113-23-36-6.ngrok-free.app" 
# query_embedding = model.encode(question)
# hits = util.semantic_search(query_embedding, embeddings, top_k=50)[0]
# cross_inp = [[preprocess_text(question), preprocess_text(corpus.iloc[hit['corpus_id']]['text'])] for hit in hits]
# cross_scores = cross_encoder.predict(cross_inp)
# for idx in range(len(hits)):
#     hits[idx]['cross-score'] = cross_scores[idx]
# hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
# best_hit = hits[0]
# top_k_cross = corpus.iloc[best_hit['corpus_id']]['text']
# print('cross-encoder')

# # Cross-encoder refinement
# print(top_k_cross)

# try:
#     response = requests.post(
#         f"{NGROK_SERVER_URL}/ask",  # Đường dẫn tới endpoint xử lý trên server ngrok
#         json={
#             "question": question,
#             "context": top_k_cross  # Gửi thông tin cần thiết
#         }
#     )
#     response_data = response.json()
    
#     answer = response_data["answer"]
#     print(answer)
# except Exception as e:
#     print(f"Error calling external server: {str(e)}")
    

In [13]:

# torch.cuda.empty_cache()
# app = Flask(__name__)
# CORS(app)

# NGROK_SERVER_URL = "https://7501-113-23-36-6.ngrok-free.app" 

# @app.route('/predict', methods=['POST'])
# def predict():
#     try:
#         # Lấy dữ liệu từ request.form
#         question = request.form.get('message')
       
#         if not question:
#             return jsonify({"error": "Missing question"}), 400

#         query_embedding = model.encode(question)
#         hits = util.semantic_search(query_embedding, embeddings, top_k=50)[0]
#         cross_inp = [[preprocess_text(question), preprocess_text(corpus.iloc[hit['corpus_id']]['text'])] for hit in hits]
#         cross_scores = cross_encoder.predict(cross_inp)
#         for idx in range(len(hits)):
#             hits[idx]['cross-score'] = cross_scores[idx]
#         hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
#         best_hit = hits[0]
#         top_k_cross = corpus.iloc[best_hit['corpus_id']]['text']
#         print('cross-encoder')
        
#         # Cross-encoder refinement
#         print(top_k_cross)
        
#         try:
#             response = requests.post(
#                 f"{NGROK_SERVER_URL}/ask",  # Đường dẫn tới endpoint xử lý trên server ngrok
#                 json={
#                     "question": question,
#                     "context": top_k_cross  # Gửi thông tin cần thiết
#                 }
#             )
#             response_data = response.json()

#             # Kiểm tra phản hồi từ server
#             if response.status_code != 200 or "answer" not in response_data:
#                 return jsonify({"answer": "Error processing the request to the external server."})
            
#             answer = response_data["answer"]
#             print(answer)
#         except Exception as e:
#             print(f"Error calling external server: {str(e)}")
#             return jsonify({"answer": "Error contacting external server."})

#         if not answer:
#             return jsonify({"answer": "Sorry, I couldn't generate an appropriate response."})
#         return jsonify({"answer": answer})

#     except Exception as e:
#         # Log lỗi và trả về phản hồi JSON lỗi
#         print(f"Error: {str(e)}")
#         return jsonify({"error": f"An error occurred: {str(e)}"}), 500

# # Start the Flask server
# if __name__ == '__main__':
#     public_url = ngrok.connect(5000)  
#     print(f"Public URL: {public_url}")
#     app.run(host='0.0.0.0', port=5000)

In [14]:
# torch.cuda.empty_cache()
# query = "trốn nghĩa vụ quân sự có đi tù không ?"
# query_embedding = model.encode(query)
# hits = util.semantic_search(query_embedding, embeddings, top_k=100)[0]
# for hit in hits[:10]:
#     score = hit['score']
#     corpus_id = hit['corpus_id']
#     print("bi_score: ", score)

#     print(str(cids[corpus_id].decode()))
#     # print(corpus.loc[(corpus['cid']) == (str(cids[corpus_id].decode())), 'text'].iloc[0])
#     print(corpus.iloc[corpus_id]['text'])
#     print(" ")

# cross_inp = [[preprocess_text(query), preprocess_text(corpus.iloc[hit['corpus_id']]['text'])] for hit in hits]

# cross_scores = cross_encoder_og.predict(cross_inp)
# for idx in range(len(hits)):
#     hits[idx]['cross-score'] = cross_scores[idx]
# hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
# for hit in hits[:10]:
#     score = hit['cross-score']
#     corpus_id = hit['corpus_id']
#     print("cross_score: ", score)

#     print(str(cids[corpus_id].decode()))
#     # print(corpus.loc[(corpus['cid']) == (str(cids[corpus_id].decode())), 'text'].iloc[0])
#     print(corpus.iloc[corpus_id]['text'])
#     print(" ")

# torch.cuda.empty_cache()

In [15]:
# import random
# from sentence_transformers import SentenceTransformer
# from sentence_transformers.evaluation import InformationRetrievalEvaluator
# from datasets import load_dataset

In [16]:
# corpus_eval = load_dataset("hiieu/legal_eval", split="corpus")
# queries_eval = load_dataset("hiieu/legal_eval", split="queries")
# relevant_docs_data = load_dataset("hiieu/legal_eval_label", split="train")
# # Convert the datasets to dictionaries
# corpus_eval = dict(zip(corpus_eval["id"], corpus_eval["text"]))  # Our corpus (cid => document)
# queries_eval = dict(zip(queries_eval["id"], queries_eval["text"]))  # Our queries (qid => question)
# relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
# for qid, corpus_ids in zip(relevant_docs_data["question_id"], relevant_docs_data["corpus_id"]):
#     qid = str(qid)
#     corpus_ids = str(corpus_ids)
#     if qid not in relevant_docs:
#         relevant_docs[qid] = set()
#     relevant_docs[qid].add(corpus_ids)

In [17]:
# import json
# import torch
# from sentence_transformers import SentenceTransformer
# from sentence_transformers.evaluation import (
#     InformationRetrievalEvaluator,
#     SequentialEvaluator,
# )
# from sentence_transformers.util import cos_sim
# from datasets import load_dataset, concatenate_datasets

# matryoshka_dimensions = [768] # Important: large to small
# matryoshka_evaluators = []
# # Iterate over the different dimensions
# for dim in matryoshka_dimensions:
#     ir_evaluator = InformationRetrievalEvaluator(
#         queries=queries_eval,
#         corpus=corpus_eval,
#         relevant_docs=relevant_docs,
#         name=f"dim_{dim}",
#         truncate_dim=dim,  # Truncate the embeddings to a certain dimension
#         score_functions={"cosine": cos_sim},
#     )
#     matryoshka_evaluators.append(ir_evaluator)

# # Create a sequential evaluator
# evaluator = SequentialEvaluator(matryoshka_evaluators)

In [18]:
# # Evaluate the model
# results = evaluator(model)
# for k,v in results.items():
#     print(k, v)

In [19]:
# model_m3 = SentenceTransformer("/kaggle/input/bge_m3_finetuned/transformers/default/1/checkpoint-5000")

In [20]:
# results = evaluator(model)
# for k,v in results.items():
#     print(k, v)

In [21]:
# import pandas as pd
# import nltk
# from nltk.tokenize import word_tokenize

# # Download the punkt tokenizer if not already downloaded
# nltk.download('punkt')

# # Load the CSV file into a DataFrame
# df = pd.read_csv('/kaggle/input/corpus-after-preprocessed-18de/corpus_after_preprocessed_18Dec.csv')  # Replace 'your_file.csv' with your actual file path

# # Specify the column name containing the text data
# text_column = 'text'  # Replace 'your_column_name' with the actual column name

# # Ensure the specified column exists in the DataFrame
# if text_column not in df.columns:
#     raise ValueError(f"Column '{text_column}' not found in the CSV file.")

# # Tokenize the text in each row of the specified column and count the tokens
# token_counts = df[text_column].dropna().apply(lambda text: len(word_tokenize(text)))

# # Calculate the maximum and mean token counts
# max_tokens = token_counts.max()
# mean_tokens = token_counts.mean()

# # Print the results
# print(f"Maximum number of tokens in a single entry: {max_tokens}")
# print(f"Mean number of tokens across all entries: {mean_tokens:.2f}")


# evaluate

In [22]:
import pandas as pd
import numpy as np

# Đọc file predict.zip

def read_predict_file(file_path):
    predictions = {}
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            qid = parts[0]
            cids = [cid.split('_')[0] for cid in parts[1:]]
            predictions[qid] = cids
    return predictions

def parse_context(text):
    result = []
    parts = text.strip("[]").split("\n")
    for part in parts:
        part = part.strip()
        if "'" in part:
            result.append(part.strip("'"))
        else:
            result.extend(part.split())
    return result

# Đọc file CSV gốc (9456 sample cuối)
def read_ground_truth(train):
    # Lấy 9456 sample cuối
    df = train.head(9456)
    
    # Tạo từ điển để tra cứu nhanh
    ground_truth = {}
    for _, row in df.iterrows():
        ground_truth[str(row['qid'])] = row['cid']
    
    return ground_truth

# Tính MRR@10
def calculate_mrr(predictions, ground_truth):
    mrr_scores = []
    for qid, label_list in ground_truth.items():
        if qid in predictions:
            predicted_cids = [str(cid) for cid in predictions[qid]]
            rank = None
            for i, cid in enumerate(predicted_cids):
                if cid in [str(label) for label in label_list]:
                    rank = i + 1
                    break
            if rank is not None:
                mrr_scores.append(1 / rank)
            else:
                mrr_scores.append(0)
        else:
            mrr_scores.append(0)
    
    return np.mean(mrr_scores)

# Đường dẫn file
predict_file = '/kaggle/input/query-ela-jin/predict.zip'  # Hoặc 'predict.txt' tùy cách bạn lưu
csv_file = '/kaggle/input/query-ela-jin/train_csv.csv'

In [23]:
# import py_vncorenlp
# rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/kaggle/input/vn-core-nlp/VnCoreNLP-master')

In [24]:
train = pd.read_csv("/kaggle/input/train-after-preprocessed-18dec/train_after_preprocessed_18Dec.csv")
train['cid'] = train['cid'].apply(lambda x: x.strip("[]"))
train['cid'] = train['cid'].astype(str)
train_subset = train.iloc[90000:95000]

In [25]:
train_subset.info()

In [26]:
from tqdm import tqdm

In [28]:
def calculate_mrr_at_k(test_data, k=50):
    """
    Calculate MRR@K based on the re-ranked document IDs.

    Parameters:
        test_data (list): List of test cases in the format [{'query': str, 'cid': int, 'top_k_cross': list}, ...].
                         Each `top_k_cross` is a list of cids (document IDs) ranked by the cross-encoder.
        k (int): Number of top documents to consider for MRR@K.

    Returns:
        float: MRR@K score for the test dataset.
    """
    mrr_score = 0.0
    total_queries = len(test_data)

    for test_case in tqdm(range(total_queries)):
        query = test_data.iloc[test_case]['question']
        top_k = 50
        query_embedding = model.encode(query)
        
        hits = util.semantic_search(query_embedding, embeddings, top_k=top_k)[0]

        top_k_bi = []
        for hit in hits:
            top_k_bi.append(str(cids[hit['corpus_id']].decode()))
        
        # cross_inp = [[query, preprocess_text(corpus.iloc[hit['corpus_id']]['context'])] for hit in hits]
        # cross_scores = cross_encoder.predict(cross_inp)
        # for idx in range(len(hits)):
        #     hits[idx]['cross-score'] = cross_scores[idx]
        # hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
        # top_k_cross = cids[corpus_id].decode()

        if len(test_data.iloc[test_case]['cid']) > 1:
            true_cid = test_data.iloc[test_case]['cid']
        
        # Calculate Reciprocal Rank (RR) for this query
        rr = 0.0
        for rank, cid in enumerate(top_k_bi, start=1):
            if cid == true_cid:
                rr = 1 / rank
                break

        mrr_score += rr

    # Compute the Mean Reciprocal Rank (MRR@K)
    mrr_score /= total_queries
    return mrr_score



In [29]:
bi_score = calculate_mrr_at_k(train_subset)

In [1]:
print("halong_embedding_finetuned")
print(f"MRR@10: {bi_score}")

halong_embedding_finetuned
MRR@10: 0.49228


In [32]:
# print("halong_embedding_finetuned")
# print(f"MRR@50: {bi_score}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

cross-encoder
Điều 8. Điều kiện kết hôn Nam, nữ kết hôn với nhau phải tuân theo các điều kiện sau đây. Nam từ đủ 20 tuổi trở lên, nữ từ đủ 18 tuổi trở lên. Việc kết hôn do nam và nữ tự nguyện quyết định. Không bị mất năng lực hành vi dân sự. Việc kết hôn không thuộc một trong các trường hợp cấm kết hôn theo quy định tại các điểm a, b, c và d khoản 2 Điều 5 của Luật này. Nhà nước không thừa nhận hôn nhân giữa những người cùng giới tính.
  Nam phải đủ từ 20 tuổi trở lên để được phép kết hôn theo quy định pháp luật. 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

cross-encoder
Phạt tiền từ 000.000 đồng đến 000.000 đồng đối với một trong các hành vi sau đây. Không tổ chức thực hiện các biện pháp phòng, chống tác hại của rượu, bia trong cơ quan, tổ chức. Không tổ chức thực hiện quy định không uống rượu, bia trong thời gian làm việc, tại nơi làm việc của cơ quan, tổ chức. Không nhắc nhở, yêu cầu chấm dứt hành vi uống, bán rượu, bia trong địa điểm không uống, bán rượu, bia thuộc quyền quản lý, điều hành. Không tổ chức thực hiện, hướng dẫn, kiểm tra, đôn đốc mọi người thực hiện đúng quy định về không được uống, không được bán rượu, bia tại địa điểm thuộc, quyền quản lý, điều hành. Phạt tiền từ 000.000 đồng đến 10.000.000 đồng đối với hành vi của người đứng đầu cơ sở kinh doanh vận tải, chủ phương tiện giao thông vận tải không thực hiện biện pháp phòng ngừa, phát hiện, ngăn chặn người điều khiến phương tiện vận tải uống rượu, bia ngay trước và trong khi tham gia giao thông.
  Uống rượu, bia khi tham gia giao thông có thể bị phạt tiền từ 000.000 đồng 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

cross-encoder
“Điều 33 Tội trốn tránh nghĩa vụ quân sự Người nào không chấp hành đúng quy định của pháp luật về đăng ký nghĩa vụ quân sự, không chấp hành lệnh gọi nhập ngũ, lệnh gọi tập trung huấn luyện, đã bị xử phạt vi phạm hành chính về hành vi này hoặc đã bị kết án về tội này, chưa được xóa án tích mà còn vi phạm, thì bị phạt cải tạo không giam giữ đến 02 năm hoặc phạt tù từ 03 tháng đến 02 năm. Phạm tội thuộc một trong các trường hợp sau đây, thì bị phạt tù từ 01 năm đến 05 năm. Tự gây thương tích hoặc gây tổn hại cho sức khỏe của mình. Phạm tội trong thời chiến. Lôi kéo người khác phạm tội. Điều 33 Tội không chấp hành lệnh gọi quân nhân dự bị nhập ngũ Người nào là quân nhân dự bị mà không chấp hành lệnh gọi nhập ngũ trong trường hợp có lệnh tổng động viên, lệnh động viên cục bộ, có chiến tranh hoặc có nhu cầu tăng cường cho lực lượng thường trực của quân đội để chiến đấu bảo vệ địa phương, bảo vệ chủ quyền lãnh thổ, thì bị phạt cải tạo không giam giữ đến 03 năm hoặc phạt tù từ 06